In [1]:
import io
import os

from google.cloud import vision

#%load_ext dotenv
#%dotenv

In [2]:
def combine_words(ocr_results, proximity_threshold=5):
    combined_results = []
    current_phrase = ""
    current_box = None
    last_word_end = None

    for i, result in enumerate(ocr_results):
        if i == 0: continue
        # Get the start (x_min) and end (x_max) of the bounding box for the current word
        vertices = ([(vertex.x, vertex.y)
                    for vertex in result.bounding_poly.vertices])
        x_min = vertices[0][0]
        x_max = vertices[1][0]

        if last_word_end is None or \
            (x_min - last_word_end) <= proximity_threshold:
            # Append to the current phrase
            current_phrase += result.description
            # Extend the current bounding box
            if current_box is None:
                current_box = result.bounding_poly
            else:
                current_box.vertices[1].x = vertices[1][0]
                current_box.vertices[2].x = vertices[2][0]
        else:
            # Start a new phrase and bounding box
            combined_results.append((current_phrase, current_box))
            current_phrase = result.description
            current_box = result.bounding_poly
        
        # Update the end position of the last word
        last_word_end = x_max

    # Don't forget to add the last phrase and box
    if current_phrase:
        combined_results.append((current_phrase, current_box))
    
    return combined_results

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '구글api jason path' 
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}

In [4]:
def detect_text(path):
    client = vision.ImageAnnotatorClient(client_options=client_options)



    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    try:
        image = vision.Image(content=content)

        response = client.text_detection(image=image)
        texts = response.text_annotations
        # print('Texts:')

        test = combine_words(texts, 5)
        # print([item[0] for item in test])
        # print([bbox[1] for bbox in test])
        
        return test
    except Exception as e:
        raise Exception(f"Error: {e}")

In [6]:
result = detect_text("C:\\Users\\user\\Desktop\\Dream\\FordA\\Test6.png")

In [ ]:
result

[('④',
  vertices {
    x: 20
    y: 39
  }
  vertices {
    x: 38
    y: 39
  }
  vertices {
    x: 38
    y: 53
  }
  vertices {
    x: 20
    y: 53
  }),
 ('종근당뼈건강프로젝트365칼마디아KBONE',
  vertices {
    x: 48
    y: 38
  }
  vertices {
    x: 53
    y: 38
  }
  vertices {
    x: 53
    y: 52
  }
  vertices {
    x: 48
    y: 52
  }),
 ('CARE',
  vertices {
    x: 59
    y: 180
  }
  vertices {
    x: 94
    y: 180
  }
  vertices {
    x: 94
    y: 190
  }
  vertices {
    x: 59
    y: 190
  }),
 ('PROJECT365',
  vertices {
    x: 100
    y: 180
  }
  vertices {
    x: 190
    y: 180
  }
  vertices {
    x: 190
    y: 190
  }
  vertices {
    x: 100
    y: 190
  }),
 ('GMP',
  vertices {
    x: 279
    y: 39
  }
  vertices {
    x: 310
    y: 39
  }
  vertices {
    x: 310
    y: 49
  }
  vertices {
    x: 279
    y: 49
  }),
 ('건강기능식품[원료명및함량]해조칼슘,산화마그네슘,산화아연,비타민D3혼합제제분말(비타민D3,아라비아검,자당,옥수수전분,중쇄중성지방,이산화규소,비타민E),비타민K,혼합제제(비타민K,아라비아검,자당),치커리뿌리추출분말,히드록시프로필메틸셀룰로스,결정셀룰로스,이산화규소,카복시메틸셀룰로스칼슘,칼슘혼합

In [ ]:
pip install openai

  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached pydantic-2.6.4-py3-none-any.whl.metadata (85 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.16.3-cp311-none-win_amd64.whl.metadata (6.6 kB)
   ---------------------------------------- 0.0/266.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/266.9 kB 1.4 MB/s eta 0:00:01
   ------------------------------ --------- 204.8/266.9 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 266.9/266.9 kB 3.3 MB/s eta 0:00:00
Using cached anyio-4.3.0-py3-none-any.

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
    )

completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "너는 OCR결과를 받아서 한국어 의미단위로 단어를 합쳐주는 Serializer야. 데이터를 받으면 해당 데이터를 의미단위로 Serialize해주고, 맞춤법이 틀린 부분이 있다면 해당하는 bbox 좌표와 함께 어디가 틀렸고, 어떻게 고쳐야하는지 알려줘"},
    {"role": "user", "content": f"데이터는 이거야. {result}"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Serialized Data:\n```\n[섭취량 및 섭취방법] 1일 1회, 1회 1포를 섭취하십시오. [섭취 시 주의사항] 특이체질, 알레르기체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오. 어린이가 함부로 섭취하지 않도록 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 일일섭취양 방법을 지도하십시오. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. 과량 섭취시 설사를 유발할 수 있습니다.\n```\n\n맞춤법 검사:\n1. "섭취하십시오.어린이가" → "섭취하십시오. 어린이가": 문장 사이에 공백이 필요합니다.\n   - {"x": 765, "y": 42, "x": 651, "y": 42, "x": 651, "y": 64, "x": 765, "y": 64}\n2. "않도록질환이" → "않도록 질환이": 단어 사이에 공백이 필요합니다.\n   - {"x": 834, "y": 76, "x": 65, "y": 76, "x": 65, "y": 98, "x": 834, "y": 98}\n3. "상담하십시오.일일섭취양" → "상담하십시오. 일일섭취양": 문장 사이에 공백이 필요합니다.\n   - {"x": 407, "y": 77, "x": 108, "y": 77, "x": 108, "y": 98, "x": 407, "y": 98}\n   \n틀린 문장의 경우, 맞춤법에 맞게 교정된 문장을 제시하였습니다. 각 문장에 대한 bounding box 정보는 시작 좌표 (x, y)와 끝 좌표 (x, y)를 통해 제공됩니다.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
    )

completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  messages=[
    {"role": "system", "content": "너는 OCR결과를 받아서 한국어 의미단위로 단어를 합쳐주는 Serializer 겸 맞춤법 검사기야. 데이터를 받으면 해당 데이터를 의미단위로 Serialize해주고, 맞춤법이 틀린 부분이 있다면 해당하는 bbox 좌표와 함께 어디가 틀렸고, 어떻게 고쳐야하는지 알려줘"},
    {"role": "user", "content": f"데이터는 이거야. {result}"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='처리된 텍스트와 맞춤법 검사 결과는 다음과 같습니다:\n\n**처리된 텍스트:**\n\n"[섭취량 및 섭취방법] 1일 1회, 1회 1포를 섭취하십시오. [섭취 시 주의사항]. 특이체질, 알레르기체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오. 어린이가 함부로 섭취하지 않도록 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 일일 섭취양 방법을 지도하십시오. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. 과량 섭취 시 설사를 유발할 수 있습니다."\n\n**맞춤법 검사 결과:**\n\n1. 잘못된 텍스트: "섭취하십시오.어린이가"  \n   수정된 텍스트: "섭취하십시오. 어린이가"  \n   문제의 bbox 좌표: `[765, 42], [651, 42], [651, 64], [765, 64]`\n\n2. 잘못된 텍스트: "않도록질환이"  \n   수정된 텍스트: "않도록 질환이"  \n   문제의 bbox 좌표: `[834, 76], [65, 76], [65, 98], [834, 98]`\n\n3. 잘못된 텍스트: "상담하십시오.일일섭취양"  \n   수정된 텍스트: "상담하십시오. 일일 섭취량"  \n   문제의 bbox 좌표: `[407, 77], [108, 77], [108, 98], [407, 98]`\n\n4. 잘못된 텍스트: "••"  \n   수정된 텍스트: - (부적절한 문자로 처리 불가)  \n   문제의 bbox 좌표: `[550, 84], [339, 84], [339, 88], [550, 88]`\n\n5. 잘못된 텍스트: "상담하십시오."  \n   수정된 텍스트: "상담하십시오."  \n   문제의 bbox 좌표: `[792, 110], [33, 110], [33, 131], [792, 131]`  \n   비고: 좌표 잘못된 부분이 있음. 다시 확인 필요.\n\n위 리스트는 입력된 데이터의 일부 문제점들을 수정하고, 오탈자나 잘못된

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
)

completion = client.chat.completions.create(
  model="gpt-4", # 모델 버전을 변경하여 시도해보세요.
  messages=[
    {
      "role": "system", 
      "content": "이 시스템은 OCR로부터 받은 한국어 텍스트를 의미 단위로 정리하고, 맞춤법을 검사하여 오류를 지적하는 역할을 합니다. OCR 데이터를 분석하여 의미 단위로 정리해주고, 맞춤법 오류가 있는 경우 정확한 수정 제안과 함께 해당 bbox 좌표를 제공합니다. 문서는 공식적인 서류로, 정확하고 공손한 언어 사용이 요구됩니다."
    },
    {
      "role": "user", 
      "content": f"데이터는 이거야. {result}"
    }
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='OCR로부터 분석한 결과, 텍스트의 본문은 다음과 같습니다.\n\n"[섭취량 및 섭취방법] 1일 1회, 1회 1포를 섭취하십시오. [섭취 시 주의사항] 특이체질, 알레르기체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오. 어린이가 함부로 섭취하지 않도록 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 일일섭취양 방법을 지도하십시오. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오. 과량 섭취시 설사를 유발할 수 있습니다."\n\n맞춤법 검사를 진행한 결과 다음과 같습니다.  \n\n1. \'[섭취 시 주의사항].\' -> 가장 가까운 bbox는 (647, 9), (741, 31)\n2. \'섭취하십시오.어린이가\' -> 가장 가까운 bbox는 (765, 42), (651, 64)\n3. \'않도록질환이\' -> 가장 가까운 bbox는 (834, 76), (65, 98)\n4. \'상담하십시오.일일섭취양\' -> 가장 가까운 bbox는 (407, 77), (108, 98)\n\n자세한 내용은 아래와 같습니다:\n\n1. "[섭취 시 주의사항]" 뒤에 있는 마침표는 제거해야 합니다.\n2. "섭취하십시오.어린이가" -> "섭취하십시오. 어린이가" 사이에 공백이 필요합니다.\n3. "않도록질환이" -> "않도록 질환이" 사이에 공백이 필요합니다.\n4. "상담하십시오.일일섭취양" -> "상담하십시오. 일일섭취양" 사이에 공백이 필요합니다.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo", # 모델 버전을 변경하여 시도해보세요.
  messages=[
    {
      "role": "system", 
      "content": "이 시스템은 OCR로부터 받은 한국어 텍스트를 의미 단위로 정리하고, 맞춤법을 검사하여 오류를 지적하는 역할을 합니다. OCR 데이터를 분석하여 의미 단위로 정리해주고, 맞춤법 오류가 있는 경우 정확한 수정 제안과 함께 해당 bbox 좌표를 제공합니다. 문서는 공식적인 서류로, 정확하고 공손한 언어 사용이 요구됩니다."
    },
    {
      "role": "user", 
      "content": f"데이터는 이거야. {result}"
    }
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='데이터는 다음과 같습니다:\n\n1. 섭취량\n2. 및 섭취방법\n3. 1일\n4. 1회,\n5. 1회\n6. 1포를 섭취하십시오.\n7. [섭취\n8. 시 주의사항].\n9. 특이체질, 알레르기체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오. 어린이가 함부로 섭취하지 않도록 질환이 있거나 의약품 복용 시 전문가와 상담하십시오. 일일 섭취량 방법을 지도하십시오.\n10. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하십시오.\n11. 과량 섭취시 설사를 유발할 수 있습니다.\n\n위 텍스트에서 오타가 발견되지 않았습니다.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo", # 모델 버전을 변경하여 시도해보세요.
  messages=[
    {
      "role": "system", 
      "content": "이 시스템은 OCR로부터 받은 한국어 텍스트를 띄어쓰기 단위로 정리하고, 맞춤법을 검사하여 오류를 지적하는 역할을 합니다. OCR 데이터를 분석하여 띄어쓰기 단위로 정리해주고, 맞춤법 오류가 있는 경우 정확한 수정 제안과 함께 해당 bbox 좌표를 제공합니다. 문서는 공식적인 서류로, 정확하고 공손한 언어 사용이 요구됩니다."
    },
    {
      "role": "user", 
      "content": f"데이터는 이거야. {result}"
    }
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="데이터는 이거야. \n\n- '[섭취량', '및', '섭취방법]'  \n- '1일', '1회,', '1회', '1포를', '섭취', '하십시오.'  \n- '[섭취', '시', '주의사항].', '특이체질,', '알레르기체질', '등은', '개인에', '따라', '과민반응을', '나타낼', '수', '있으므로', '원료를', '확인한', '후', '섭취하십시오.어린이가', '함부로', '섭취하지', '않도록질환이', '있거나', '의약품', '복용', '시', '전문가와', '상담하십시오.일일섭취양', '방법을', '지도하십시오.'  \n- '••', '이상사례', '발생', '시', '섭취를', '중단하고', '전문가와', '상담하십시오.', '과량', '섭취시', '설사를', '유발할', '수', '있습니다.'  \n\n문장부호가 누락된 부분이 있습니다. 올바른 문장부호를 추가해야 합니다.\x0b", role='assistant', function_call=None, tool_calls=None)


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo", # 모델 버전을 변경하여 시도해보세요.
  messages=[
    {
      "role": "system", 
      "content": "이 시스템은 OCR로부터 받은 한국어 텍스트를 띄어쓰기 단위로 정리하고, 맞춤법을 검사하여 오류를 지적하는 역할을 합니다. OCR 데이터를 분석하여 띄어쓰기 단위로 정리해주고, 맞춤법 오류가 있는 경우 정확한 수정 제안과 함께 해당 bbox 좌표를 제공합니다. 문서는 공식적인 서류로, 정확하고 공손한 언어 사용이 요구됩니다."
    },
    {
      "role": "user", 
      "content": f"데이터는 이거야. {result}"
    }
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="데이터는 이거야. \n\n[('섭취량',  vertices { x: 7, y: 9 }), ('및', vertices { x: 87, y: 9 }), ('섭취방법', vertices { x: 119, y: 9 }), ('1일', vertices { x: 220, y: 9 }), ('1회,', vertices { x: 262, y: 9 }), ('1회', vertices { x: 312, y: 9 }), ('1포를', vertices { x: 356, y: 9 }), ('섭취', vertices { x: 417, y: 9 }), ('하십시오.', vertices { x: 459, y: 9 }), ('섭취', vertices { x: 558, y: 9 }), ('시', vertices { x: 616, y: 9 }), ('주의사항', vertices { x: 647, y: 9 }), ('특이체질,', vertices { x: 754, y: 9 }), ('알레르기체질', vertices { x: 853, y: 9 }), ('등은', vertices { x: 100, y: 42 }), ('개인에', vertices { x: 153, y: 42 }), ('따라', vertices { x: 225, y: 42 }), ('과민반응을', vertices { x: 277, y: 42 }), ('나타낼', vertices { x: 390, y: 42 }), ('수', vertices { x: 463, y: 42 }), ('있으므로', vertices { x: 496, y: 42 }), ('원료를', vertices { x: 587, y: 42 }), ('확인한', vertices { x: 660, y: 42 }), ('후', vertices { x: 733, y: 42 }), ('섭취하십시오.', vertices { x: 765, y: 42 }), ('어린이가', vertices { x: 651, y: 42 }), ('함부로

띄어쓰기 bbox, 맞춤법 따로

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key='API Key'
)


# 띄어쓰기 단위로 텍스트를 정리합니다.
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "너는 OCR로부터 받은 한국어 텍스트를 띄어쓰기 단위로 정리하는 역할이야. OCR 데이터를 분석하여 띄어쓰기 단위로 정리해주고, bbox 좌표를 제공해줘. OCR에서 분석된 내용을 바꾸지는 마."
    },
    {
      "role": "user",
      "content": f"데이터는 이거야. {result}"
    }
  ]
)

# 첫 번째 단계의 출력을 저장합니다.
ocr_text = completion.choices[0].message.content
print(ocr_text)


데이터는 이거야. [('④', vertices {
  x: 20
  y: 39
}
vertices {
  x: 38
  y: 39
}
vertices {
  x: 38
  y: 53
}
vertices {
  x: 20
  y: 53
}
), ('종근당 뼈건강 프로젝트 365 칼마디아 KBONE', vertices {
  x: 48
  y: 38
}
vertices {
  x: 53
  y: 38
}
vertices {
  x: 53
  y: 52
}
vertices {
  x: 48
  y: 52
}
), ('CARE', vertices {
  x: 59
  y: 180
}
vertices {
  x: 94
  y: 180
}
vertices {
  x: 94
  y: 190
}
vertices {
  x: 59
  y: 190
}
), ('PROJECT 365', vertices {
  x: 100
  y: 180
}
vertices {
  x: 190
  y: 180
}
vertices {
  x: 190
  y: 190
}
vertices {
  x: 100
  y: 190
}
), ('GMP', vertices {
  x: 279
  y: 39
}
vertices {
  x: 310
  y: 39
}
vertices {
  x: 310
  y: 49
}
vertices {
  x: 279
  y: 49
}
), ('건강기능식품 [원료명및함량] 해조칼슘, 산화마그네슘, 산화아연, 비타민 D3 혼합제제 분말 (비타민 D3, 아라비아검, 자당, 옥수수전분, 중쇄중성지방, 이산화규소, 비타민E), 비타민 K, 혼합제제 (비타민 K, 아라비아검, 자당), 치커리뿌리 추출분말, 히드록시프로필메틸셀룰로스, 결정셀룰로스, 이산화규소, 카복시메틸셀룰로스칼슘, 칼슘혼합제제 (탄산칼슘, 변성전분), 스테아린산마그네슘, 스테아린산, 글리세린지방산에스테르, 바실러스나토균농축분말 [섭취량및섭취방법] 1일 1회, 1회 2정을 물과 함께 섭취하십시오. [섭취시주의사항] 특이체질,

In [ ]:
# 맞춤법 검사를 진행합니다.
completion_for_correction = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "너는 한국어 텍스트의 맞춤법을 검사하고 수정 제안을 하는 역할이야. 텍스트를 분석하여 맞춤법 오류가 있는 경우 모든 부분의 정확한 수정 제안과 함께 해당 bbox 좌표를 제공해줘. 공식적인 말투로 말해줘."
    },
    {
      "role": "user",
      "content": ocr_text
    }
  ]
)

# 맞춤법 검사 결과를 출력합니다.
correction = completion_for_correction.choices[0].message.content
print(correction)


주어진 텍스트의 맞춤법을 검사한 결과, 몇 가지 오류를 발견했습니다. 먼저 "건강기능식품"과 "맞춤제제" 사이에 띄어쓰기 오류가 있습니다. 또한 "특이체질"이 아닌 "특이 체질"로 수정하는 것이 적절합니다. 마지막으로 "용기 안의 방습제는 섭취하지 마십시오"에서 "마십시오" 대신 "마십시요"로 수정하는 것이 더 자연스러울 것입니다. 

수정 후의 전체 텍스트는 다음과 같습니다:
"건강기능식품 [원료명 및 함량] 해조칼슘, 산화마그네슘, 산화아연, 비타민 D3 혼합제제 분말 (비타민 D3, 아라비아검, 자당, 옥수수전분, 중쇄 중성지방, 이산화규소, 비타민E), 비타민 K, 혼합제제 (비타민 K, 아라비아검, 자당), 치커리뿌리 추출분말, 히드록시프로필메틸셀룰로스, 결정셀룰로스, 이산화규소, 카복시메틸셀룰로스칼슘, 칼슘 혼합제제 (탄산칼슘, 변성전분), 스테아린산마그네슘, 스테아린산, 글리세린지방산에스테르, 바실러스나토균농축분말 [섭취량 및 섭취방법] 1일 1회, 1회 2정을 물과 함께 섭취하십시오. [섭취시 주의사항] 특이 체질, 알레르기 체질 등은 개인에 따라 과민반응을 나타낼 수 있으므로 원료를 확인한 후 섭취하십시오. 고칼슘혈증이 있거나 의약품 복용 시 전문가와 상담하시기 바랍니다. 이상사례 발생 시 섭취를 중단하고 전문가와 상담하시기 바랍니다. 항응고제 등 복용 시 전문가와 상담하십시요. 용기 안의 방습제는 섭취하지 마십시요. [보관 시 주의사항] 습기가 적고 직사광선을 받지 않는 실온에 보관하시고, 어린이의 손에 닿지 않도록 주의하시기 바랍니다."

수정 제안 사항이 반영된 bbox 좌표는 아래와 같습니다:
1. ('건강기능식품', (340, 31), (66, 31), (66, 38), (340, 38))
2. ('특이체질', (130, 583), (158, 583), (158, 594), (130, 594))
3. ('마십시오', (592, 962), (640, 962), (640, 972), (592, 972))
